In [14]:
import mido
from mido import MidiFile, Message, MidiTrack
import torch
import os
import numpy as np
import time
import random

In [15]:
majScale = np.array([0,2,4,5,7,9,11])
minScale = np.array([0,2,3,5,7,8,10])

In [16]:
def getAllNotesMajor(root):# Get all used notes in major scale of root=root
    notes = []
    intervals = [2,2,1,2,2,2,1]
    
    while root > 24: #bring down to lowest used octave
        root -= 12
    
    n = root
    notes.append(n)
    while n < 84: #up to higherst used note
        for i in intervals:
            n += i
            notes.append(n)   
    return notes    


def getRangeMajor(notes, low, high): # get range of voice
    lowIndex = notes.index(low)
    highIndex = notes.index(high)
    
    return notes[lowIndex:highIndex+1]   

In [17]:
class Piece: # Entire baseline model compostion - composed of 4 voices soprano, alto, tenor, bass (SATB)
    def __init__(self, barNum=16, root=60):# 16 bars in C major
        self.root = root # root note
        self.allNotes = getAllNotesMajor(self.root) # all notes on major scale
        self.barNum = barNum # number of bars
        
        self.soprano = Voice(self.allNotes,60,84,speed=8) # SATB
        self.alto = Voice(self.allNotes,48,72)
        self.tenor = Voice(self.allNotes,36,60)
        self.bass = Voice(self.allNotes,24,48)
          
        self.notes = np.empty([0,4]) #notes output
        
        self.pieceChords = [] # chords
        
        self.chords = np.array([ # common classical C major chords
            [ 0,  4,  7,  0],# I
            [ 2,  5,  9,  2],# ii
            [ 4,  7, 11,  4],# iii
            [ 5,  9, 0,  5],# IV
            [ 7, 11, 2,  7],# V
            [ 9, 0, 4,  9],# vi
            [11, 2, 5, 11],# vii dim
            [ 2,  5,  9, 0],# ii7
            [ 5,  9, 0, 4],# IVmaj7
            [ 7, 11, 2, 5],# V7
            [11, 2, 5, 9]])# vii7 half-dim
        
    def generateSoprano(self): # generate soprano line
        self.soprano.generateLine(self.soprano.speed*self.barNum)
        
    def generateAlto(self): # generate alto line from chords
        self.alto.generateChordLine(self.pieceChords)
        
    def generateTenor(self): # see alto
        self.tenor.generateChordLine(self.pieceChords)
        
    def generateBass(self): # see alto
        self.bass.generateChordLine(self.pieceChords)
        
        
    
    def chooseChord(self, sopNote): # choose a fitting chord for soprano note
        while sopNote >= 12:
            sopNote -= 12
        
        goodChords = np.empty([0,4])
        
        for chord in self.chords:
            if (chord==sopNote).sum() > 0:
                goodChords = np.append(goodChords,[chord],axis=0)
        
        chosenChord = goodChords[random.randint(0,len(goodChords)-1)]
        chosenChord = np.sort(np.unique(chosenChord))
        
        i = 12
        chordNotes = chosenChord
        while i < 120:
            chordNotes = np.append(chordNotes, chosenChord+i)
            i += 12
        
        return(chordNotes)
    
    def getChords(self): # select all chords in piece
        for i, note in enumerate(self.soprano.notes):
            if i % 2 == 0:
                sopNote = note[0]
                chord = self.chooseChord(sopNote)
                self.pieceChords.append(chord)
                
    def generateLines(self): # generate all SATB lines and joins them - entire baseline model
        self.generateSoprano()
        self.getChords()
        self.generateAlto()
        self.generateTenor()
        self.generateBass()
        self.joinLines()
        
        return self.notes
        
    def insertLine(self, starting, inserted, startIndex, skipIndex): # join 2 lines
        base = np.copy(starting)
        ins = np.copy(inserted)
        
        for i,note in enumerate(ins):
            base = np.insert(base, (i*skipIndex)+startIndex, [note], axis=0)
            
        return base
        
    def joinLines(self): # join all SATB lines
        #self.notes = np.copy(self.soprano)
        self.notes = self.insertLine(self.soprano.notes, self.alto.notes, 1, 3)
        self.notes = self.insertLine(self.notes, self.tenor.notes, 2, 4)
        self.notes = self.insertLine(self.notes, self.bass.notes, 3, 5)
        

In [21]:
class Voice: # individual voices
    def __init__(self, allNotes, lowNote, highNote, jump=3, speed=4, time=1024, velocity=64):
        self.range = getRangeMajor(allNotes,lowNote,highNote) #available ntoes
        self.jump = jump #maximum pitch interval between notes
        self.speed = speed #note length i.e. 4 for quarter, 8 for eighth etc.
        self.time = time #song speed
        self.velocity = velocity #note volume
        self.notes = np.empty([0,4]) #notes output
        self.lowNote = lowNote # lowest note
        self.highNote = highNote # highest note
        self.allNotes = allNotes # all notes in scale
            
        self.duration = self.time / self.speed # time between notes
        
        
    def randomStartNote(self): # generate random first note (for soprano)
        note = random.choice(self.range)
        self.notes = np.append(self.notes,np.array([[note, self.velocity, 0, self.duration]]),axis=0)
        
        
    def randomJump(self): # generate random next note (for soprano)
        lastNote = self.notes[len(self.notes)-1][0] # find last played note
        lastIndex = self.range.index(lastNote)
        
        newIndex = -1
        while newIndex < 0 or newIndex >= len(self.range): # stay in range
            newIndex = lastIndex + random.randint(-self.jump,self.jump)
            
        newNote = self.range[newIndex]
        self.notes = np.append(self.notes,np.array([[newNote, self.velocity, self.duration, self.duration]]),axis=0)
        
        
    def generateLine(self, length): # generate random line (for soprano)
        self.randomStartNote()
        
        for n in range(length-1):
            self.randomJump()
            
            
    def clearNotes(self):
        self.notes = np.empty([0,4])
        
    def getChordNotes(self, chordNotes): # get useful notes from all chord notes
        chordNotes = chordNotes[chordNotes >= self.lowNote]
        chordNotes = chordNotes[chordNotes <= self.highNote]
        return chordNotes
    
    def chooseStartChordNote(self, chordNotes): # choose random note in chord
        note = random.choice(chordNotes)
        self.notes = np.append(self.notes,np.array([[note, self.velocity, 0, self.duration]]),axis=0)
        
    def chooseChordNote(self,chordNotes): # choose suitable next note in chord
        lastNote = self.notes[len(self.notes)-1][0] # find last played note
        
        chordNotes = chordNotes[chordNotes >= lastNote - (self.jump*2)]
        chordNotes = chordNotes[chordNotes <= lastNote + (self.jump*2)]
        newNote = random.choice(chordNotes)
        
        self.notes = np.append(self.notes,np.array([[newNote, self.velocity, 0, self.duration]]),axis=0)
        
    def generateChordLine(self, chords): # generate A/T/B lines
        
        firstChord = self.getChordNotes(chords[0])
        self.chooseStartChordNote(firstChord)
        
        for c in chords[1:]:
            chord = self.getChordNotes(c)
            self.chooseChordNote(chord)    
        
        
        

In [22]:
p = Piece()
p.generateLines()
np.savetxt("baseline.csv", p.notes, delimiter=",")